<a href="https://www.kaggle.com/code/pietrocaforio/unimodal-ct-training-kaggle?scriptVersionId=199214851" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 345 (delta 80), reused 75 (delta 29), pack-reused 208 (from 1)
Receiving objects: 100% (345/345), 5.18 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (202/202), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

from sklearn.utils.class_weight import compute_class_weight

### Train ResNet model

In [7]:
#https://github.com/mathiaszinnen/focal_loss_torch/tree/main
!pip install focal_loss_torch

In [8]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [9]:
from focal_loss.focal_loss import FocalLoss

def train(model,config, run_name=None):
    wandb.init(
        # set the wandb project where this run will be logged
        project="unimodal_ct_training",
        name = run_name,
        # track hyperparameters and run metadata
        config=config
    )
    if config["class_weights"] is not None: 
        config["class_weights"] = torch.tensor(config["class_weights"], dtype=torch.float).to(device) 
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
    if config["focal_loss"] is not None:
        
        criterion = FocalLoss(gamma = config["focal_loss"])
    else:
        criterion = nn.CrossEntropyLoss(weight = config["class_weights"])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = config["reduce_lr_factor"], patience = config["patience"])
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=config["early_stop_patience"], verbose=True)
    
    # Training loop
    num_epochs = config["epochs"]
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_correct = 0
        total = 0
        correct_per_class = [0, 0, 0]  # For G1, G2, G3
        total_per_class = [0, 0, 0]  # For G1, G2, G3
        for batch in train_loader:
            frames = batch['frame'].float().to(device)
            labels = batch['label'].long().to(device)
            
            optimizer.zero_grad()
            outputs = model(frames)
            if config["focal_loss"]:
                softmax = torch.nn.Softmax(dim=-1)
                loss = criterion(softmax(outputs.logits), labels)
            else:
                loss = criterion(outputs.logits, labels)

            
            _, predicted = torch.max(outputs.logits, 1)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()
            total += labels.size(0)
            running_loss += loss.item()
            
            # Calculate accuracy per class
            for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                total_per_class[i] += (labels == i).sum().item()

        train_accuracy = 100 * train_correct / total
        class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")
        wandb.log({"Train Accuracy": train_accuracy, "Train loss": running_loss/len(train_loader), "G1_TrainAcc":class_accuracy[0], "G2_TrainAcc":class_accuracy[1], "G3_TrainAcc":class_accuracy[2]})

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        # Initialize counters for each class (G1, G2, G3)
        correct_per_class = [0, 0, 0]  # For G1, G2, G3
        total_per_class = [0, 0, 0]  # For G1, G2, G3

        with torch.no_grad():
            for batch in val_loader:
                frames = batch['frame'].float().to(device)
                labels = batch['label'].long().to(device)

                outputs = model(frames)
                
                if config["focal_loss"]:
                    softmax = torch.nn.Softmax(dim=-1)
                    loss = criterion(softmax(outputs.logits), labels)
                else:
                    loss = criterion(outputs.logits, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Calculate accuracy per class
                for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                    correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                    total_per_class[i] += (labels == i).sum().item()
        scheduler.step(val_loss)
        # Compute total accuracy and per-class accuracy
        total_accuracy = 100 * correct / total
        class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
        print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
        print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")
        # log metrics to wandb
        wandb.log({"Total Accuracy": total_accuracy, "Validation Loss": val_loss/len(val_loader), "G1_Acc":class_accuracy[0], "G2_Acc":class_accuracy[1], "G3_Acc":class_accuracy[2]})
        early_stopping(val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
    wandb.finish()  

In [10]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification
from sklearn.model_selection import GroupKFold #For crossvalidation

In [11]:
train_dataset = UnimodalCTDataset(split='all',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )


In [12]:
#print(f"Training set stats:{train_dataset.stats()}")
#print(f"Validation set stats:{val_dataset.stats()}")

In [13]:
"""
labels = []
for sample in train_dataset:
    labels.append(sample["label"])
labels = np.array(labels)
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
"""

'\nlabels = []\nfor sample in train_dataset:\n    labels.append(sample["label"])\nlabels = np.array(labels)\nclass_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)\n'

In [14]:
#print(class_weights)

In [15]:
#train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/oversampling57patientscptacpda/processed_oversampling/" )
#val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/oversampling57patientscptacpda/processed_oversampling/")

#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [16]:
#print(f"Training set stats:{train_dataset.stats()}")
#print(f"Validation set stats:{val_dataset.stats()}")

In [17]:
#total = train_dataset.stats()["length"]
#most_frequent = max(train_dataset.stats()["class_frequency"].values())
#freq_dict = train_dataset.stats()["class_frequency"]
#target_volume_depth= {}
#for index in freq_dict.keys():
#    target_volume_depth[index] = int((total/3 ) * most_frequent / freq_dict[index])
#print(target_volume_depth)

### Resnet-50

In [18]:
#model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
#model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [19]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [20]:
"""
config={
    "learning_rate": 1e-4,
    "architecture": "microsoft/resnet-50 new1",
    "epochs": 100,
    "weight_decay": 1e-4,
    "reduce_lr_factor": 0.2,
    "patience": 10,
    "class_weights": class_weights
    }
train(model, config, run_name = config["architecture"])
"""

'\nconfig={\n    "learning_rate": 1e-4,\n    "architecture": "microsoft/resnet-50 new1",\n    "epochs": 100,\n    "weight_decay": 1e-4,\n    "reduce_lr_factor": 0.2,\n    "patience": 10,\n    "class_weights": class_weights\n    }\ntrain(model, config, run_name = config["architecture"])\n'

### Resnet-18

In [21]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [23]:

config={
    "learning_rate": 1e-7,
    "architecture": "microsoft/resnet-18 NOOVERSAMPLING FOCALLOSS",
    "epochs": 2,
    "weight_decay": 1e-6,
    "reduce_lr_factor": 0.25,
    "patience": 20,
    "early_stop_patience": 30,
    "class_weights": None,
    "focal_loss": 2
    }




### Resnet-34

In [24]:
#model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
#model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [25]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [26]:
k_folds = 3
batch_size = 32
# Initialize the k-fold cross validation
#group fold in order to group indices by patient_id so that we don't introduct frames of the same patient in the train set and test set
gkf = GroupKFold(n_splits=k_folds) 
patient_ids = [info.split("_")[0] for info in train_dataset.items]
indices = list(range(len(patient_ids)))
# Loop through each fold
for fold, (train_idx, test_idx) in enumerate(gkf.split(indices, groups=patient_ids)):
    print(f"Fold {fold + 1}")
    print("-------")
    # Define the data loaders for the current fold
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(train_idx),
    )
    val_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(test_idx),
    )
    config["architecture"] = f"{config['architecture']} fold:{fold + 1}"    
    train(model, config, run_name = config["architecture"])


wandb: Currently logged in as: pietro-caforio (pietro-caforio-politecnico-di-milano). Use `wandb login --relogin` to force relogin


Fold 1
-------


wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241004_163725-lwkl57ud
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-18 NOOVERSAMPLING FOCALLOSS fold:1
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/lwkl57ud


Epoch 1, Loss: 0.45799362616262573
Validation Loss: 0.48499068532671247, Total Accuracy: 45.73%
Accuracy per class - G1: 1.87%, G2: 62.10%, G3: 7.55%
Validation loss decreased (inf --> 16.974674).  Saving model ...
Epoch 2, Loss: 0.44501148445018823
Validation Loss: 0.4640001348086766, Total Accuracy: 51.55%
Accuracy per class - G1: 5.61%, G2: 71.45%, G3: 1.42%
Validation loss decreased (16.974674 --> 16.240005).  Saving model ...


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁█
wandb:     G1_TrainAcc ▁▁
wandb:          G2_Acc ▁█
wandb:     G2_TrainAcc ▁█
wandb:          G3_Acc █▁
wandb:     G3_TrainAcc █▁
wandb:  Total Accuracy ▁█
wandb:  Train Accuracy ▁█
wandb:      Train loss █▁
wandb: Validation Loss █▁
wandb: 
wandb: Run summary:
wandb:          G1_Acc 5.60748
wandb:     G1_TrainAcc 5
wandb:          G2_Acc 71.44686
wandb:     G2_TrainAcc 89.80695
wandb:          G3_Acc 1.41509
wandb:     G3_TrainAcc 8.90125
wandb:  Total Accuracy 51.54545
wandb:  Train Accuracy 56.33546
wandb:      Train loss 0.44501
wandb: Validation Loss 0.464
wandb: 
wandb: 🚀 View run microsoft/resnet-18 NOOVERSAMPLING FOCALLOSS fold:1 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/lwkl57ud
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&

Fold 2
-------
Epoch 1, Loss: 0.4302621565867161
Validation Loss: 0.42428878801209585, Total Accuracy: 59.07%
Accuracy per class - G1: 1.72%, G2: 91.89%, G3: 10.79%
Validation loss decreased (inf --> 14.850108).  Saving model ...
Epoch 2, Loss: 0.4103792080844658
Validation Loss: 0.4105180101735251, Total Accuracy: 59.80%
Accuracy per class - G1: 1.72%, G2: 91.89%, G3: 13.33%
Validation loss decreased (14.850108 --> 14.368130).  Saving model ...


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁
wandb:     G1_TrainAcc █▁
wandb:          G2_Acc ▁▁
wandb:     G2_TrainAcc ▁█
wandb:          G3_Acc ▁█
wandb:     G3_TrainAcc ▁█
wandb:  Total Accuracy ▁█
wandb:  Train Accuracy ▁█
wandb:      Train loss █▁
wandb: Validation Loss █▁
wandb: 
wandb: Run summary:
wandb:          G1_Acc 1.72414
wandb:     G1_TrainAcc 3.50877
wandb:          G2_Acc 91.89189
wandb:     G2_TrainAcc 86.73759
wandb:          G3_Acc 13.33333
wandb:     G3_TrainAcc 8.11688
wandb:  Total Accuracy 59.79945
wandb:  Train Accuracy 58.21575
wandb:      Train loss 0.41038
wandb: Validation Loss 0.41052
wandb: 
wandb: 🚀 View run microsoft/resnet-18 NOOVERSAMPLING FOCALLOSS fold:1 fold:2 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/5qk6b5e3
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wa

Fold 3
-------
Epoch 1, Loss: 0.40787800369055377
Validation Loss: 0.41397938472884044, Total Accuracy: 57.16%
Accuracy per class - G1: 6.25%, G2: 95.07%, G3: 6.19%
Validation loss decreased (inf --> 14.489278).  Saving model ...
Epoch 2, Loss: 0.38839045512503473
Validation Loss: 0.4156081974506378, Total Accuracy: 57.43%
Accuracy per class - G1: 10.94%, G2: 94.91%, G3: 6.44%
EarlyStopping counter: 1 out of 30


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁█
wandb:     G1_TrainAcc ▁█
wandb:          G2_Acc █▁
wandb:     G2_TrainAcc ▁█
wandb:          G3_Acc ▁█
wandb:     G3_TrainAcc ▁█
wandb:  Total Accuracy ▁█
wandb:  Train Accuracy ▁█
wandb:      Train loss █▁
wandb: Validation Loss ▁█
wandb: 
wandb: Run summary:
wandb:          G1_Acc 10.9375
wandb:     G1_TrainAcc 7.17489
wandb:          G2_Acc 94.91256
wandb:     G2_TrainAcc 88.25155
wandb:          G3_Acc 6.43564
wandb:     G3_TrainAcc 10.81594
wandb:  Total Accuracy 57.42935
wandb:  Train Accuracy 61.44743
wandb:      Train loss 0.38839
wandb: Validation Loss 0.41561
wandb: 
wandb: 🚀 View run microsoft/resnet-18 NOOVERSAMPLING FOCALLOSS fold:1 fold:2 fold:3 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/4iqg48h2
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_trai